## <span style="color:grey">Initialization</span>

In [1]:
try:
    mode
except Exception as e:
    %run "KiteConnect_Library.ipynb"  
    logging.debug(e)
    mode = "Algo"
    papertrade = True


##  <span style="color:green">Trading Strategy </span>

In [92]:
# ====== Tradescript Wrapper =======
# Variables
OPEN = 0
CLOSE = 0
HIGH = 0
LOW = 0
VOLUME = 0
BBT = 0
BBM = 0
BBB = 0
AroonUp = 0
AroonDown = 0
iMACD = 0
iMACDhist = 0
iMACDsignal = 0
UP = 0
DOWN = 1
SD = 0
iRSI = 0 
atr = 0
STATE = "SELL"
sma20 = 0
sma50 = 0
buy_df = pd.DataFrame()
sell_df = pd.DataFrame()
fastk = 0
fastd = 0

# Methods
REF = lambda df, i: df.shift(i)
TREND_UP = lambda a: ROC(a, 4) >= 0.1
TREND_DOWN = lambda a: ROC(a, 4) <= -0.1

#TREND_UP = lambda high, low: ((REF(high,1) >= REF(high,2)) & (REF(low,1) >= REF(low,2)) & (ROC(CLOSE) >1) )
#TREND_DOWN =  lambda high, low: ((REF(high,1) <= REF(high,2)) & (REF(low,1) <= REF(low,2))  & (ROC(CLOSE) <1))

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)
profit = 0
stoploss = 0
squareoff = 0

from talib import CDLLONGLEGGEDDOJI, CDLMARUBOZU, CDLLONGLINE, CDLDARKCLOUDCOVER, CDLPIERCING, CDLBELTHOLD, CDLCLOSINGMARUBOZU

class algoTrade:
    
    def __init__(s, price):
        #logging.debug(price.tail(10))
        global OPEN, CLOSE, HIGH, LOW, VOLUME, BBT, BBM, BBB, AroonDown, AroonUp
        global iMACD, iMACDhist, iMACDsignal, SD, iRSI, sma20, sma50, fastk, fastd
        OPEN = price['open']
        CLOSE = price['close']
        HIGH = price['high']
        LOW = price['low']
        VOLUME = price['volume']
        BBT, BBM, BBB = BBANDS( CLOSE, 20,1.90,1.90,0)
        BBT, BBM, BBB = BBANDS( CLOSE, 20,1.90,1.90,0)
        iMACD, iMACDsignal, iMACDhist = MACDEXT(CLOSE, fastperiod=12, slowperiod=26, signalperiod=9,  fastmatype=1, slowmatype=1,signalmatype=1)
        SD = STDDEV(CLOSE)
        sma20 = SMA(CLOSE, timeperiod=20)
        sma50 = SMA(CLOSE, timeperiod=50)
        fastk, fastd = STOCHF(HIGH, LOW, CLOSE)
    
    # Long Strategies
    def long_bb(self):
        return pd.DataFrame( CROSSOVER(CLOSE, BBB) & ( CROSSOVER(fastk, fastd)| ((REF(fastk,1) <=20)&(fastk>20) )), columns=["buy"] )
     
    # Short Strategies
    def short_bb(self):
        return pd.DataFrame(CROSSOVER(BBT, CLOSE) & (CROSSOVER(fastd, fastk) | ((REF(fastk,1) >=80)&(fastk<80) ) ), columns=["sell"])
    
    
    def long_pa(self):
        temp_df = 0
        temp_df =   (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == 100)
        #temp_df = temp_df | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == 100)
        #temp_df = temp_df | (CDLPIERCING(OPEN, HIGH, LOW, CLOSE) == 100)
        #temp_df = temp_df | (CDLHAMMER(OPEN, HIGH, LOW, CLOSE) != 0)
        temp_df = temp_df | ((CDLMORNINGSTAR(OPEN, HIGH, LOW, CLOSE) == 100) & (CDLMORNINGDOJISTAR(OPEN, HIGH, LOW, CLOSE) == 100))
        temp_df = temp_df | (CDLMARUBOZU(OPEN, HIGH, LOW, CLOSE) == 100)# | (CDLCLOSINGMARUBOZU(OPEN, HIGH, LOW, CLOSE) ==100)
        temp_df = temp_df | (CDLBELTHOLD(OPEN, HIGH, LOW, CLOSE) == 100)
        
        temp_df = REF(TREND_DOWN(CLOSE),1) & temp_df
        
        return pd.DataFrame(temp_df, columns=["buy"])
    def short_pa(self):
        temp_df = 0
        
        temp_df =   (CDLENGULFING(OPEN, HIGH, LOW, CLOSE) == -100)
        #temp_df = temp_df | (CDLHARAMI(OPEN, HIGH, LOW, CLOSE) == -100)
        #temp_df = temp_df | (CDLDARKCLOUDCOVER(OPEN, HIGH, LOW, CLOSE) == -100)
        #temp_df = temp_df | (CDLDOJISTAR(OPEN, HIGH, LOW, CLOSE) != 0)
        temp_df = temp_df | ((CDLEVENINGSTAR(OPEN, HIGH, LOW, CLOSE) == -100) & (CDLEVENINGDOJISTAR(OPEN, HIGH, LOW, CLOSE) == -100))
        temp_df = temp_df | (CDLMARUBOZU(OPEN, HIGH, LOW, CLOSE) == -100)# | (CDLCLOSINGMARUBOZU(OPEN, HIGH, LOW, CLOSE) == -100)
        #temp_df = temp_df | (CDLBELTHOLD(OPEN, HIGH, LOW, CLOSE) == -100)
        
        temp_df = REF(TREND_UP(CLOSE),1) & temp_df
        
        return pd.DataFrame( temp_df , columns=["sell"])
  
    def tradeDecision(self):
        
        #buy  = self.long_pa()   
        #sell = self.short_pa()
        
        buy = self.long_bb()
        sell = self.short_bb()
        
        buy['low'] = LOW
        sell['high'] = HIGH
        buy['close'] = -1 * CLOSE
        sell['close'] = CLOSE

        buy = buy[buy['buy']]
        sell = sell[sell['sell']]
        
        temp_df = pd.DataFrame(buy.close)
        
        temp_df = pd.concat([buy.close, sell.close])
        #temp_df.append(buy.close)
        #temp_df = temp_df.append(sell.close)
        print(temp_df.sort_index())
        return (buy,sell)
    
    
    # do not set timestamp for live trading
    def tradeBot(self, symbol, timestamp=dt.datetime.now().replace(second=0, microsecond=0)):
        global STATE, profit, squareoff, stoploss
        
        timeNow = timestamp        
        delT1 = timeNow - dt.timedelta(minutes = 1)
        delT2 = timeNow - dt.timedelta(minutes = 2)

        price = CLOSE[-1]
        trigger = CLOSE[-1]
        stoploss = price - 2 * SD[-1] 
        squareoff = 3 * SD[-1] 

        check_trade = lambda x, dt1, dt2: True if (x.shape[0] > 0) and ((x.index[-1]==dt1) or (x.index[-1]==dt2)) else False
        print_trade = lambda x: logging.debug(x) if (x.empty != True) else False
        
        buy = pd.DataFrame()
        sell = pd.DataFrame()
        
        if STATE == "BUY":
            buy  = self.long_pa()
            buy = buy[buy['buy']]
                #print_trade(buy)
                #STATE = "SELL"
        elif STATE == "SELL":
            sell = self.short_pa()
            sell = sell[sell['sell']]

        '''
        
        if (timeNow.time() > dt.time(11,30)) and (timeNow.time() < dt.time(14,0)):
            # Mostly Choppy period# Mostly Trending period
            if STATE == "BUY":
                buy  = self.long_pa()
                buy = buy[buy['buy']]
                #print_trade(buy)
                #STATE = "SELL"
            elif STATE == "SELL":
                sell = self.short_pa()
                sell = sell[sell['sell']]
                #print_trade(sell)   
                #STATE = "BUY" 
        else:
            # Mostly Trending period
            if STATE == "BUY":
                buy  = self.long_pa()
                buy = buy[buy['buy']]
                #print_trade(buy)
                #STATE = "SELL"
            elif STATE == "SELL":
                sell = self.short_pa()
                sell = sell[sell['sell']]
                #print_trade(sell)   
                #STATE = "BUY"    '''
            
        if check_trade(buy, delT1, delT2):            
            logger.debug("buy:"+str(buy.index[-1]))
            
            ##### Order Placement #####
            #buy_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            buy_slm(symbol, price, stoploss, quantity=1)
            STATE = "SELL"
            return -1 * price
            
        elif check_trade(sell, delT1, delT2):                
            logger.debug("sell:"+str(sell.index[-1]))
            
            ##### Order Placement #####
            #sell_bo(symbol, price, trigger, stoploss, squareoff, quantity=1)
            sell_slm(symbol, price, stoploss, quantity=1)
            STATE = "BUY"
            return price
        
        return 0

#portfolio = pd.DataFrame(downloadlist)

if mode == "Algo":
    x = 'BAJAJ-AUTO'
    
    toDate = dt.datetime(2019,4,18,15,15)
    fromDate = toDate - dt.timedelta(days=1)
    STATE = "BUY"
    logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame([x])
    papertrade = True
    
    temp_data = getData(x, fromDate, toDate, "minute")
    
    a = algoTrade(temp_data)
    
    buy, sell = a.tradeDecision()
    
    #print(buy, sell)
    
    fig = createPlot(x)
    plot_set1(temp_data)
    
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)

date
2019-04-18 10:08:00   -3057.00
2019-04-18 10:11:00   -3054.80
2019-04-18 10:38:00    3058.00
2019-04-18 10:42:00    3063.00
2019-04-18 10:59:00    3067.00
2019-04-18 11:04:00    3071.00
2019-04-18 12:10:00    3084.60
2019-04-18 12:28:00    3087.05
2019-04-18 12:35:00    3088.15
2019-04-18 12:45:00   -3083.40
2019-04-18 13:14:00   -3084.00
2019-04-18 13:20:00   -3083.40
2019-04-18 13:25:00   -3079.85
2019-04-18 13:28:00   -3078.55
2019-04-18 13:59:00   -3082.10
2019-04-18 14:34:00   -3082.00
2019-04-18 14:40:00   -3077.35
2019-04-18 14:50:00   -3077.00
Name: close, dtype: float64
This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [47]:
from talib import CDLBELTHOLD
df = CDLBELTHOLD(OPEN,HIGH,LOW,CLOSE)

#df[df!=0]

In [66]:
if mode == "Algo":
    x = 'WIPRO'
    
    toDate = dt.datetime(2019,4,18,15,15)
    fromDate = toDate - dt.timedelta(days=3)
    STATE = "BUY"
    logger.setLevel(logging.DEBUG)
    portfolio = pd.DataFrame([x])
    papertrade = True
    
    temp_data = getData(x, fromDate, toDate, "minute")
    
    a = algoTrade(temp_data)
    
    buy, sell = a.tradeDecision()
    
    fig = createPlot(x)
    plot_set1(temp_data)
    
    buy_df = annotateBuySell(buy, "Buy")
    sell_df = annotateBuySell(sell, "Sell")
    plotData(x, True)

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]



In [67]:
gc.collect()

20

In [93]:
portfolio = ['CIPLA']
if mode == "Algo":
    for x in portfolio:
        toDate = dt.datetime(2019,4,18,15,15)
        fromDate = toDate - dt.timedelta(days = 2, hours=4)
        STATE = "BUY"
        logger.setLevel(logging.DEBUG)
        portfolio = pd.DataFrame([x])
        papertrade = True

        backtest(portfolio, fromDate, toDate)
        gc.collect()

    #temp_data = getData(x, fromDate = fromDate, toDate=toDate, freq="minute")
    #fig = createPlot(x)
    #plot_set1(temp_data)
    #annotateBuySell(buy_df[buy_df['symbol']==x], "Buy")
    #annotateBuySell(sell_df[sell_df['symbol']==x], "Sell")
    #plotData(x, True)
    #gc.collect()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



CIPLA: 3.2, (0.5702 %)


- Lunch Time: 11:00 am to 2:00 pm => Choppy Market
- Bollinger band ZONE: for breakout

Choppy market Trading:
- Identify stocks with highest SD and trade using bollinger

Trending Market Trading:
- MACD or price action trading based on candle stick


No new open position after 3:00. Only close open positions

before 9:15: Go long based on one day candle
9:15 - 11:30: MACD or Stochastic+BB: One trigger, and atleast one confirmation
11:30 - 2:00: No trade untill and unless there is a golden cross or death cross
2:00 - 2:45: Go long if price is smaller than VWAP, Go short if price is higher than VWAP
2:00 - 2:45: Go long or short only if price crosses SMA(200)
